# Wordly
## Experiment 2016-12-21

Initial experiments with n-gram model - setting up architecture. 

In [66]:
from __future__ import print_function, division

import os.path
import random

In [58]:
import sys; sys.path.append('../../src')
print('importing wp (and nltk)...')
import wp
reload(wp)
print('done')
dir(wp)

importing wp (and nltk)...
done


['__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '__path__',
 'ngram',
 'rnn',
 'split']

In [59]:
from wp import ngram, rnn
reload(wp.ngram)
reload(wp.rnn)
model = ngram.NgramModel(2) # bigram
model

<wp.ngram.NgramModel instance at 0x000000001F249DC8>

In [54]:
dir(model)

['__doc__',
 '__init__',
 '__module__',
 '__str__',
 '_d',
 'get_random',
 'increment',
 'load',
 'n',
 'predict',
 'save',
 'train']

## Load Training Data

In [48]:
trainfile = 'data/all-train.txt'

with open(trainfile, 'rb') as f:
    s_train = f.read()
    s_train = s_train.strip()
    s_train = s_train.lower()
    f.close()

## Explore Training Data

In [55]:
# show some samples of the training text
nchars = len(s_train)
nsamples = 5
nskip = int(nchars / nsamples)
for i in range(nsamples):
    s = s_train[i*nskip:i*nskip+200]
    s = s.replace('\n', ' ').strip()
    print(s)
    print()

the project gutenberg ebook of phantastes, by george macdonald  this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.  yet is it a little window, that loo

o and come in affright as though tossed back and forth between the swords of the uhlans and the fusillade of the brigades of kempt, best, pack, and rylandt; the worst of hand-to-hand conflicts is the

, the sight of birds, tender shadows, agitated branches, and a soul made of sweetness, of faith, of candor, of hope, of aspiration, and of illusion.  cosette had left the convent when she was still al

prepossessing exterior.  coarse paper, coarsely folded--the very sight of certain missives is displeasing.  the letter which basque had brought was of this sort.  it smelled of tobacco.  marius recogn

s.  the ring of the fort drew him with stronger fascination during that hot august weather.  standing, or as his headmaster would have said, "mooning" by the gate, and looking into that enclose

In [69]:
# split some text into sentences and show a few - 
# this shows how the text was split up into train, validate, and test sets.
sentences = wp.split.get_sentences(s_train[0:50000]) # use first 50k instead of all 6mb (slow)
nsamples = 5
l = random.sample(sentences, nsamples)
print('\n\n'.join(l))

i crossed the rivulet, and accompanied it, keeping the footpath on its right bank, until it led me, as i expected, into the wood.

it caused a new sensation.

indeed, there were more instruments at work about her than there could have been sparks in her.

it is a mere matter of relation.

but the door of a little cupboard in the centre especially attracted my interest, as if there lay the secret of this long-hidden world.


## Train / Load Models

In [34]:
#. put in a pandas table? use to store results also? 

mlist = [
    ['n-gram (n=2)', 'ngram-model-basic-n-2', ngram.NgramModel, {'n':2}],
    ['n-gram (n=3)', 'ngram-model-basic-n-3', ngram.NgramModel, {'n':3}],
    ['rnn', 'rnn', rnn.RnnModel, {}],
]

for m in mlist:
    modelname = m[0]
    modelfile = 'models/' + m[1] + '.pickle'
    modelclass = m[2]
    modelparams = m[3]

    # load existing model, or create, train, and save one
    if os.path.isfile(modelfile):
        print("load model: " + modelfile)
        model = modelclass.load(modelfile) # static method
    else:        
        model = modelclass(**modelparams) # constructor

        #print("read file: " + trainfile)
        #f = open(trainfile, 'rb')
        #s = f.read()
        #s = s.strip()
        #s = s.lower()
        #f.close()

        print("train model")
        model.train(s)

        print("save model: " + modelfile)
        model.save(modelfile)

    m.append(model)
    
print("done")

load model: models/ngram-model-basic-n-2.pickle
load model: models/ngram-model-basic-n-3.pickle
load model: models/rnn.pickle
done


## Test Models

In [ ]:
infile = 'data/all-test.txt'

f = open(infile, 'rb')
s = f.read()
s = s.strip()
s = s.lower()
f.close()

In [17]:
# walk through file with tokenizer
#print s[:1000]

from nltk import tokenize
words = tokenize.word_tokenize(s)
wlist = [words[i:] for i in range(n)]
i=0
for words in zip(*wlist):
    prompt = words[:-1]
    y_actual = words[-1]
    y_predict = m.predict(prompt)
    print words, y_predict
    i+=1
    if i>20: break

('greville', 'macdonald') None
('macdonald', '.') this
('.', 'september') the
('september', '1905.') ,
('1905.', 'phantastes') None
('phantastes', 'a') ,
('a', 'faerie') little
('faerie', 'romance') bulwarks
('romance', 'phantastes') ,
('phantastes', 'from') ,
('from', 'their') the
('their', 'fount') own
('fount', 'all') None
('all', 'shapes') the
('shapes', 'deriving') ,
('deriving', ',') None
(',', 'in') and
('in', 'new') the
('new', 'habiliments') york
('habiliments', 'can') of
('can', 'quickly') not


NameError: name 'stop' is not defined